In [ ]:
import wandb

wandb.init(project="a2o-cnn", entity="tnapier")

wandb.config = {
  "learning_rate": 0.001,
  "epochs": 50,
  "batch_size": 16
}

In [3]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set data paths
train_data_path = "D:\\PhD\\a20data"

# Set image transformations
train_transforms = transforms.Compose([
    transforms.Resize((432, 288)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

val_transforms = transforms.Compose([
    transforms.Resize((432, 288)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load data
train_dataset, val_dataset = train_test_split(train_data_path, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Load pre-trained model
model = torchvision.models.resnet50(pretrained=True)

# Modify last layer to fit number of classes in your dataset
num_classes = len(train_dataset.classes)
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

# Move model to device
model = model.to(device)

# Set loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Fine-tune model
num_epochs = 50

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Training
    model.train()
    train_loss = 0.0
    train_corrects = 0
    
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        train_corrects += torch.sum(preds == labels.data)
        
    train_loss = train_loss / len(train_dataset)
    train_acc = train_corrects.double() / len(train_dataset)
    
    print(f"Train loss: {train_loss:.4f}, Train accuracy: {train_acc:.4f}")
    
    # Validation
    model.eval()
    val_loss = 0.0
    val_corrects = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            val_corrects += torch.sum(preds == labels.data)
        
    val_loss = val_loss / len(val_dataset)
    val_acc = val_corrects.double() / len(val_dataset)

    print(f"Val loss: {val_loss:.4f}, Val accuracy: {val_acc:.4f}")

ModuleNotFoundError: No module named 'requests'